# ElectroMKM class import and object instantiation

In [ ]:
import sys
sys.path.insert(0, "../../electromkm/") 
from electromkm import electroMKM

The demo system deal with the Hydrogen Evolution Reaction (HER) with random values.

In [ ]:
model = electroMKM('HER',
                   'rm.mkm', 
                   'g.mkm', 
                    t_ref=298)

# Model exploration


To investigate the characteristics of the system under study, several attributes can be easily inspected to check general information like number of elementary reactions, energetics, reaction network, etc.

In [ ]:
print(model)

### Defined species in the system

N.B. H(e) is used to define H+ + e-.

In [ ]:
model.v_f.shape == model.v_matrix.shape

In [ ]:
model.NC_sur

In [ ]:
model.NC_gas

In [ ]:
model.species_tot

### Visualize Gibbs energetics of the system

Reaction types: 'ads'=adsorption
                'des'=desorption
                'sur'=surface reaction. 
The suffix "+e" means that that elementary reaction is a charge-transfer step.

In [ ]:
model.df_gibbs

### Stoichiometric matrix of the reaction network

In [ ]:
model.df_system

### Stoichiometric vector of the global reactions

In [ ]:
model.species_tot

In [ ]:
model.v_global

In [ ]:
model.stoich_numbers

model.stoich_numbers tells us that the first elementary reaction must be multiplied by two and summed up to the second one in order to get the global reaction. This is useful for checking the thermodynamic consistency of the developed models.

# Microkinetic runs and Tafel plot

Up to now, it is possible to run steady state runs via the electroMKM.kinetic_run() function.
The main inputs that must be provided are the applied overpotential and the pH of the electrolyte solution.
The output of the function is a Python dictionary containing information related to the performed simulation.

In [ ]:
model.set_ODE_params(t_final=10000)

In [ ]:
model.dh_barrier

### Steady state simulation

In [ ]:
import numpy as np
exp = model.kinetic_run(-0.2, 7, initial_sur_coverage=[1,0.0])
#exp = model.kinetic_run(-0.2, 7,jac=True)

The first check should be about the reaching of the steady state. This means that all the derivatives in the ODE must equal to zero. These values can be easily visualized with the key 'ddt':

In [ ]:
import matplotlib.pyplot as plt
time = exp['solver'].t
y = exp['solver'].y[0,:]
plt.plot(time, y)

Once steady state conditions have been checked, the solution can be easily analyzed. the main output consists of steady state surface coverage and reaction rate in term of current density.

In [ ]:
exp['theta']

In [ ]:
exp['MASI']

Negative current density means reduction is occurring, while positive values means that reaction is evolving in the opposite direction. Values of current density are stored in mA cm-2.

In [ ]:
exp['j_HER']

### Tafel plot

In [ ]:
import numpy as np
n = np.array([-0.3,-0.2,-0.1,0,0.1,0.2,0.3])
print(n)

In [ ]:
model.tafel_plot("HER", n ,7, jac=True)

### Check for the analytical Jacobian matrix

In order to check if the implemented analytical Jacobian is correct or not, we run the same simulation with and without the analytical Jacobian: If the solutions are the same, the Jacobian is correct. If not, it means that the wrong Jacobian drives the system away from the correct solution. 

#### Simulation without analytical Jacobian

In [ ]:
exp1 = model.kinetic_run(-0.1, 7, jac=False)

#### Simulation with analytical Jacobian

In [ ]:
exp2 = model.kinetic_run(-0.1, 7, jac=True)

Observe the difference in the CPU time required to integrate the system without and with the Jacobian!

In [ ]:
import numpy as np
model.degree_of_rate_control("HER", "R1", -0.2, 7, gas_composition=np.array([1]))